<a href="https://colab.research.google.com/github/benreaves/fantasia-sunset/blob/master/fantasia_sunset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
def hl(elevation=7.11,azimuth=285.39):
  """ elevation is in degrees, azimuth measured CW from N
  location is -122.01597, 37.335986
  from https://gml.noaa.gov/grad/solcalc/
  returns H/L the ratio of the height of parking structure wall
  to the length from the westernmost pole of Fantasia patio
  to the base of the parking structure wall,
  assuming the wall is constant height and runs due north/south.
  TODO: make the input a time-of-day if el and az are not given.
  In that case get el and az from pyephem.
  My guess is that H is around 10ft and L is around 30ft
  
  """
  # radians
  el = elevation*math.pi/180; az=azimuth*math.pi/180
  return math.tan(el) / math.sin(3.14*2 - az)

hl(7.11, 285.39),hl(15.16,286.2)
# not getting results I like! the second one should be lower

(0.12948710550987938, 0.2824102795735398)

# When will the shadow cross a table at Fantasia?
It crosses when
tan(e) = (H/L)*sin(d)

where H is the height of the wall, and L is the distance to the pole.
https://photos.app.goo.gl/bjDWVQVTZN7GshD29

So we need to know H/L. Easier: take a photo to measure one set of d&e that work. I have several (see "Fantasia Shadow"). I can get az&el from  https://gml.noaa.gov/grad/solcalc/

location is -122.01597, 37.335986

Sunset behind the parking structure is at 20220806T1929-0700. Shadow crossing the pole is 20220712T1903-0700. This is easier to see, but the former is more useful in the hot summer

From https://gml.noaa.gov/grad/solcalc/
The first one yields az=285.39 degrees, el=7.11. The second one: az=286.2, el=15.16 this is more measureable. el is e, but az is measured CW from N, whereas d is measured CCW from N. So, 360=az+d


we have az and d, want H/L.
H/L = tan(e)/sin(d)
H/L = tan(el) / sin(360-az)
= 0.12933

How about the second one? I expect H to be lower because I'm sitting instead of on the floor. If I were standing as tall as the wall, el would be 0




In [ ]:
!pip install Skyfield pynverse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pynverse: filename=pynverse-0.1.4.4-py3-none-any.whl size=6363 sha256=9b35d68f4438791282e88f98ad5527b1229e68dda5a5d02d26b0fdb8e4568447
  Stored in directory: /root/.cache/pip/wheels/42/d4/98/c957a0dc68d88ace74b11435dae2b4fa55bfb1342a4e6b551b
Successfully built pynverse


In [44]:
from skyfield.api import N,S,E,W, wgs84, load
import datetime, pytz, numpy

def azel(dt=None, HL=hl(), ):
  if not dt:
    dt = datetime.datetime(dtn.year,dtn.month,dtn.day,12,0,0,0,pytz.timezone('America/Los_Angeles'))
  ts = load.timescale()
  t = ts.from_datetime(dt)
  ephem = load('de421.bsp')
  sun = ephem['sun']
  earth = ephem['earth']

  # https://rhodesmill.org/pyephem/quick.html
  # https://rhodesmill.org/skyfield/positions.html

  fantasia = earth + wgs84.latlon(37.335986 * N, 122.01597 * W, elevation_m=30)

  #astro = fantasia.at(t).observe(sun).apparent().altaz()
  #app = astro.apparent()

  alt, az, distance = fantasia.at(t).observe(sun).apparent().altaz()
  return az.radians, alt.radians
  # return alt.dstr(), az.dstr()

def f(minutes):
  """ we want to minimize this function.
  minutes is the number of minutes past 5pm on 2023 april 29, no, today!
  Sometimes minutes is an numpy.ndarray. why?
  """
  print("minutes", minutes, type(minutes))
  if isinstance(minutes, numpy.ndarray):
    minutes = minutes[0]
  print("minutes", minutes)
  dtn = datetime.datetime.now()
  dt5pm = datetime.datetime(dtn.year,dtn.month,dtn.day,17,0,0,0,
                            pytz.timezone('America/Los_Angeles'))
  dt = dt5pm + datetime.timedelta(minutes=minutes)
  az, el = azel(dt)
  #print(az,el)
  HL = 0.12948710550987938 # hl()
  cost = HL * math.sin(az) - math.tan(el)
  return cost


In [50]:
from pynverse import inversefunc
import numpy
#f(0), f(1), f(2), f(10), f(100)
x = inversefunc(f, y_values=0, domain=[0,4*60])
print(x, "minutes after 5pm")
x = float(x)
dtn = datetime.datetime.now()
dt5 = datetime.datetime(dtn.year,dtn.month,dtn.day,17,0,0,0,
                            pytz.timezone('America/Los_Angeles'))
dtshadow = dt5 + datetime.timedelta(minutes=x)
print(dtshadow)

minutes 180.0 <class 'float'>
minutes 180.0
minutes 60.0 <class 'float'>
minutes 60.0
minutes 0.0 <class 'float'>
minutes 0.0
minutes 240.0 <class 'float'>
minutes 240.0
minutes 240.0 <class 'float'>
minutes 240.0
minutes 0.0 <class 'float'>
minutes 0.0
minutes 60.0 <class 'float'>
minutes 60.0
minutes 180.0 <class 'float'>
minutes 180.0
minutes 134.16408 <class 'float'>
minutes 134.16408
minutes 105.83592301872001 <class 'float'>
minutes 105.83592301872001
minutes 155.7730818582702 <class 'numpy.float64'>
minutes 155.7730818582702
minutes 155.75090825269982 <class 'numpy.float64'>
minutes 155.75090825269982
minutes 155.71005871028757 <class 'numpy.float64'>
minutes 155.71005871028757
minutes 155.70980156637555 <class 'numpy.float64'>
minutes 155.70980156637555
minutes 155.7098038708906 <class 'numpy.float64'>
minutes 155.7098038708906
minutes 155.7097992618605 <class 'numpy.float64'>
minutes 155.7097992618605
minutes [155.70980157] <class 'numpy.ndarray'>
minutes 155.70980156637555
15

In [45]:
import datetime, pytz
dtn = datetime.datetime.now()
dt = datetime.datetime(dtn.year,dtn.month,dtn.day,11,21,0,0,pytz.timezone('America/Los_Angeles'))
dt

datetime.datetime(2023, 4, 30, 11, 21, tzinfo=<DstTzInfo 'America/Los_Angeles' LMT-1 day, 16:07:00 STD>)